## Load and preprocess data

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from transformers import DataCollatorWithPadding, get_scheduler
from datasets import load_dataset, Dataset, DatasetDict
from tqdm.auto import tqdm
import evaluate

# Loads PAP datasets
datasets_path = '../datasets/pap/train-dev-test-split/binary'
train_df = pd.read_csv(f'{datasets_path}/train.csv')
dev_df = pd.read_csv(f'{datasets_path}/dev.csv')
test_df = pd.read_csv(f'{datasets_path}/test.csv')

Loads and preprocess concreteness ratings:

In [ ]:
# We got this dataset for concreteness of 40k words (https://pubmed.ncbi.nlm.nih.gov/24142837/) from https://web.stanford.edu/class/linguist278/data/
# Load concreteness ratings
concreteness_df = pd.read_csv('../datasets/concreteness/Concreteness_ratings_Brysbaert_et_al_BRM.csv')
concreteness_df.head(2)

# Map and normalize conreteness ratings
word_to_concreteness_score_map = dict()
for idx, row in concreteness_df.iterrows():
    row = row.to_dict()
    
    # Normalizing to a scale of 0 to 1
    word_to_concreteness_score_map[row['Word']] = row['Conc.M']/5.0 

Define helper functions to get concreteness scores:

In [ ]:

def get_concreteness_score(word):
    """
    Get the concreteness score of a word based on the Concreteness Ratings dataset.
    """
    # If the word is not found in the dataset, return a default score of 0.5
    return round(word_to_concreteness_score_map.get(word, 0.5), 3)

def calculate_text_concreteness_sequence(text):
    """
    Calculate the concreteness score for a given text.
    """
    words = nltk.word_tokenize(text)
    concreteness_scores = [get_concreteness_score(word) for word in words]
    concreteness_scores = " ".join([str(i) for i in concreteness_scores])
    # Take the average concreteness score of all words in the text
    return concreteness_scores

Load PAP datasets:

In [ ]:
# Add concreteness scores for the every sequence
train_df['concreteness_score_sequence'] = train_df.text.apply(calculate_text_concreteness_sequence)
dev_df['concreteness_score_sequence'] = dev_df.text.apply(calculate_text_concreteness_sequence)
test_df['concreteness_score_sequence'] = test_df.text.apply(calculate_text_concreteness_sequence)

# Load PAP datasets with Concreteness Scores  
raw_datasets = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)
})

In [ ]:
class ModellingExperiments:
    COLUMNS_TO_KEEP = ['label', 'input_ids', 'token_type_ids', 'attention_mask']

    def __init__(self, model_name, dataset, batch_size, learning_rate):
        self.model_name = model_name
        self.dataset = dataset
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
        self.cols_to_keep = set(COLUMNS_TO_KEEP)
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        self.data_collator = DataCollatorWithPadding(self.tokenizer)
        self.model.to(self.device)
        
    def tokenize_sentence_with_concreteness_score(self, item):
        # We also tried using the concreteness score for the whole sentence as a feature input
        # To implement that, we changed the source code of transformers library and changed the classification head manually 
        # So that we can accomodate that extra feature, but this method of encoding concreteness score sequence was giving better score
        # Hence, we are using this only in the final experiments. You can check that experiment out in the following notebook:
        # modelling/adept/experiments/FinalModellingWithConcretenessScore[DeBERTa] - ADEPT
        return self.tokenizer(item['sentence2'], item['concreteness_score_sequence'], truncation=True)
        
    def tokenize_sentence(self, item):
        # Normal tokenization
        return self.tokenizer(item['sentence2'], truncation=True)
        
    def add_strategy_to_tokenizer_function_map(self):
        # Mapping between strategy and the tokenization functions defined above
        # Strategy refers to whether we are using normal tokenization or whether we want to do paired tokenization of 
        # both input sentence and the sequence of concreteness score for that sentence
        self.strategy_to_tokenizer_function_map = dict()
        self.strategy_to_tokenizer_function_map['normal_finetuning'] = self.tokenize_sentence_with_concreteness_score
        self.strategy_to_tokenizer_function_map['concreteness_score_addition'] = self.tokenize_sentence
        
    def prepare_dataset(self, strategy):
        # Here, we wull tokenize the dataset based on the strategy we are planning to use
        self.strategy = strategy
        self.add_strategy_to_tokenizer_function_map()
        self.tokenized_dataset = self.dataset.map(self.strategy_to_tokenizer_function_map[self.strategy], batched=True)
        current_cols = set(list(self.tokenized_dataset['train'].features.keys()))
        self.tokenized_dataset = self.tokenized_dataset.remove_columns(list(current_cols - self.cols_to_keep))
        self.tokenized_dataset = self.tokenized_dataset.rename_column("label", "labels")
        self.tokenized_dataset = self.tokenized_dataset.with_format("torch")

    def prepare_dataloaders(self):
        self.train_dataloader = DataLoader(self.tokenized_dataset['train'], batch_size=self.batch_size, shuffle=True, collate_fn=self.data_collator)
        self.validation_dataloader = DataLoader(self.tokenized_dataset['validation'], batch_size=self.batch_size, collate_fn=self.data_collator)
        self.test_dataloader = DataLoader(self.tokenized_dataset['test'], batch_size=self.batch_size, collate_fn=self.data_collator)

    def setup_optimizer(self, num_epochs):
        # Setting up optimizer and learning rate scheduler
        self.num_epochs = num_epochs
        self.optimizer = AdamW(self.model.parameters(), lr=self.learning_rate)
        self.num_training_steps = self.num_epochs*len(self.train_dataloader)
        self.learning_rate_scheduler = get_scheduler("linear", optimizer=self.optimizer, num_warmup_steps=0, num_training_steps=self.num_training_steps)
    
    def train_model(self):
        # Training the Model
        self.model.train()
        progress_bar = tqdm(range(self.num_training_steps))
        for epoch in range(self.num_epochs):
            for batch in self.train_dataloader:
                batch = {k:v.to(self.device) for k, v in batch.items()}
                outputs = self.model(**batch)
                loss = outputs.loss
                # calculating gradients
                loss.backward()
                # optimizing weights
                self.optimizer.step()
                # updating learning rate
                self.learning_rate_scheduler.step()
                # flushing gradients
                self.optimizer.zero_grad()
                # updating progress bar
                progress_bar.update(1)

    def initialize_metrics(self):
        # Initializing evaluation metrics
        self.accuracy = evaluate.load('accuracy')
        self.precision = evaluate.load('precision')
        self.recall = evaluate.load('recall')
        self.f1 = evaluate.load('f1')
        self.roc_auc =  evaluate.load("roc_auc")
        self.metrics = [self.accuracy, self.precision, self.recall, self.f1]
                
    def eval_model(self, dataloader):
        # Evaluating the model on different dataloaders
        self.initialize_metrics()
        self.model.eval()
        for batch in dataloader:
            batch = {k: v.to(self.device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = self.model(**batch)
            # Extract logits and predictions
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
        
            # Apply softmax to convert logits to probabilities
            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            
            # Extract probabilities for the positive class
            positive_probabilities = probabilities
        
            # Update metrics for accuracy, precision, recall, and F1
            for metric in self.metrics:
                metric.add_batch(predictions=predictions, references=batch['labels'])
        
            # Update ROC AUC metric
            self.roc_auc.add_batch(prediction_scores=positive_probabilities, references=batch['labels'])
        
        # # Compute metrics for accuracy, precision, recall, and F1
        self.eval_dict = {}
        self.eval_dict.update(self.accuracy.compute())
        self.eval_dict.update(self.precision.compute(average="macro"))
        self.eval_dict.update(self.recall.compute(average="macro"))
        self.eval_dict.update(self.f1.compute(average="macro"))
        self.eval_dict.update(self.roc_auc.compute(average="macro"))     

## Run Experiments

Defining parameters on which we will run the experiments

In [ ]:
# Initialize model parameters
model_name_list = ["facebook/bart-base", "microsoft/deberta-base"]
num_epochs_list = [1, 2, 3, 4, 5]
strategies_list = ["normal_finetuning", "concreteness_score_addition"]

Defining static arguments

In [ ]:
kw_args = {
    'dataset' = raw_datasets,
    'batch_size' = 32,
    'learning_rate' = 3e-5,
}

### Experiments loop

In [ ]:
result_list = list()
for model_name in model_name_list:
    
    # Setting Model Name
    kw_args["model_name"] = model_name
    
    # Initializing ModellingExperiments Object
    modelling_obj = ModellingExperiments(**kw_args)
    
    for strategy in strategies_list:
        
        # Preparing dataset for a specific strategy
        modelling_obj.prepare_dataset(strategy=strategy)
        
        # Preparing data loaders
        modelling_obj.prepare_dataloaders()
            
        # Training loop
        for num_epochs in num_epochs_list:
            # Initializing dictionary for storing results
            result_dict = dict()
            result_dict["model_name"] = model_name
            result_dict["strategy"] = strategy
            #result_dict["train_dataset_type"] = train_dataset_type
            result_dict["num_epochs"] = num_epochs
            print("Model Training with the following Configurations: {}".format(result_dict))
            
            # For a specic num_epochs variable, we are setting up the optimizers
            modelling_obj.setup_optimizer(num_epochs=num_epochs)
            
            # Now, we are training the model
            modelling_obj.train_model()
            
            # Now, we will evaluate the model on validation dataset
            modelling_obj.eval_model(modelling_obj.validation_dataloader)
            
            # Storing results on validation set
            for k, v in modelling_obj.eval_dict.items():
                result_dict["validation_{}".format(k)] = v
            print("Validation Set Results: {}".format(modelling_obj.eval_dict))
            
            # Now, we will evaluate the model on test dataset
            modelling_obj.eval_model(modelling_obj.test_dataloader)
            
            # Storing results on test set
            for k, v in modelling_obj.eval_dict.items():
                result_dict["test_{}".format(k)] = v
            
            print("Test Set Results: {}".format(modelling_obj.eval_dict))
            
            # Storing all the results in the results_list
            result_list.append(result_dict)

In [ ]:
results_df = pd.DataFrame(result_list)
results_df.head()

In [ ]:
results_df.to_csv('../../results/FinalResultsPAP.csv', index=False)